In [1]:
import random
import csv
import os
import pickle
import sys
os.chdir('Resources/')

In [2]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [3]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [4]:
sys_rsa_2_public_key = load_key("17_SYS_RSA_2_Public_Key.pkl")
sys_rsa_2_public_key

(4192637885330971573, 4774294288111246121)

In [5]:
input_filename = '22_(SYS)_CC_Predicted_User_Output_Data.csv'

In [6]:
def encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(char), e, n) for char in message]
    return encrypted_message

def encrypt_csv_file(input_filename, public_key):
    with open(input_filename, 'r') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])
    
    encrypted_data = encrypt(data, public_key)
    
    return encrypted_data

encrypted_data = encrypt_csv_file(input_filename, sys_rsa_2_public_key)
print(encrypted_data, '\n', len(encrypted_data))

def decrypt(private_key, encrypted_message):
    d, n = private_key
    decrypted_message = ''.join([chr(pow(char, d, n)) for char in encrypted_message])
    return decrypted_message

su_rsa_2_private_key = load_key("16_SU_RSA_2_Private_Key.pkl")

decrypted_data = decrypt(su_rsa_2_private_key, encrypted_data)
print(decrypted_data)

[524658259528873529, 4498258305198829921, 524658259528873529, 4249860195149345262, 1164770017514057496, 3636838611358563201, 4592127022199445741, 4498258305198829921, 2848419555841521786, 2300939831866961911, 1164770017514057496, 519337717827213692, 524658259528873529, 4498258305198829921, 4249860195149345262, 1164770017514057496, 2848419555841521786, 4249860195149345262, 4498258305198829921, 2300939831866961911, 1164770017514057496, 519337717827213692, 4498258305198829921, 524658259528873529, 524658259528873529, 1164770017514057496, 3564282447330381440, 524658259528873529, 4498258305198829921, 4325096859872585300, 1164770017514057496, 3636838611358563201, 4249860195149345262, 4498258305198829921, 3636838611358563201, 1164770017514057496, 3636838611358563201, 2848419555841521786, 4498258305198829921, 2300939831866961911, 1164770017514057496, 2300939831866961911, 2300939831866961911, 4498258305198829921, 4249860195149345262, 1164770017514057496, 4249860195149345262, 3564282447330381440,

In [7]:
def encrypt_data_with_aes(data, key):
    encrypted_data = "".join(chr(ord(c) ^ key) for c in data)
    return encrypted_data

In [8]:
def encrypt_aes_key_with_rsa(aes_key, public_key):
    e, n = public_key
    return power(aes_key, e, n)

In [9]:
def encrypt_csv_file(input_filename, public_key):
    aes_key = random.randint(1, 255) 

    with open(input_filename, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])

    encrypted_data = encrypt_data_with_aes(data, aes_key)
    encrypted_aes_key = encrypt_aes_key_with_rsa(aes_key, public_key)

    with open("23_(SYS)_CC_Transmitted_User_Output_Data.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([encrypted_aes_key])
        writer.writerow([encrypted_data])

    return encrypted_aes_key, encrypted_data

In [10]:
encrypted_aes_key, encrypted_data = encrypt_csv_file(input_filename, sys_rsa_2_public_key)
print("File encrypted successfully. Data saved in '22_(SYS)_CC_Predicted_User_Output_Data.csv'.")

print(encrypted_aes_key, encrypted_data, len(encrypted_data))

File encrypted successfully. Data saved in '22_(SYS)_CC_Predicted_User_Output_Data.csv'.
1401078831196967303 [F[\D^SFRYD_[F\DR\FYD_F[[DX[F]D^\F^D^RFYDYYF\D\XFYDR[FYD^RF]DSRFX^D]S 69


In [11]:
sys_rsa_1_private_key = load_key("16_SYS_RSA_1_Private_Key.pkl")
sys_rsa_1_private_key

(551453761721239577, 12056529777636162823)

In [12]:
sys_rsa_1_public_key = load_key("16_SYS_RSA_1_Public_Key.pkl")
sys_rsa_1_public_key

(800039276376817193, 12056529777636162823)

In [13]:
def split_message(message, max_size):
    encoded = message.encode()
    chunks = [encoded[i:i + max_size] for i in range(0, len(encoded), max_size)]
    return [int.from_bytes(chunk, 'big') for chunk in chunks]

In [14]:
def sign_message(message, private_key, n):
    d, n = private_key
    max_bytes = (n.bit_length() // 8) - 1 
    message_chunks = split_message(message, max_bytes)
    
    signatures = [power(chunk, d, n) for chunk in message_chunks] 
    return signatures

In [15]:
signatures = sign_message(str(encrypted_aes_key) + '\n' + encrypted_data, sys_rsa_1_private_key, sys_rsa_1_public_key[1])
signatures

[10157239781934559752,
 10062977358364607205,
 5568479385692153325,
 9093427784715901898,
 9594345489230289981,
 1307713079321390238,
 7727664801510162456,
 2923087285957354114,
 9076400092781076890,
 5014189973543546288,
 8774351438908640735,
 2597567152988309471,
 8637281600472874339]

In [16]:
with open("23_(SYS)_CC_Transmitted_User_Output_Data.csv", 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([signatures])